In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import random

In [ ]:
selected_tasks = [
    "task_0004", "task_0006", "task_0034", "task_0037", "task_0111", "task_0088",  # Object Interaction
    "task_0048", "task_0049", "task_0100", "task_0112",                          # Tool and Rotation
    "task_0026", "task_0029", "task_0071",                                       # Containment/Access
    "task_0001", "task_0003", "task_0023"                                        # Sequential Actions
]
command_line_dict ={}
for task in selected_tasks:
    command_line_dict[task] = []

In [ ]:
full_task2command = [
    ["task_0004", "Pick up the small block on the left and move it to the right", ['Pick up the block', 'Pick up the small block', 'Drop the small block','Put down the block']], # Object Interaction
    ["task_0006", "Use the gripper to push the small block from left to right", ['Lift up the gripper', 'Move the block from left to right', 'Push the block from left to right']],
    ["task_0100", "Stir the beaker with a glass rod", ['Pick up the rod', 'Grab the rod', 'Move the rod in circle', 'Drop the rod']],
    ['task_0111', "Put the object with the corresponding shape into the corresponding hole", ['Pick up the block', 'Pick up the sphere', 'Drop the ball']],
    ['task_0088', "Pick up and place an object with obstacles", ['Pick up the block', 'Lift up the ball', 'Move the ball']],

    ['task_0048', "Rotate the steering wheel 90 degrees clockwise", ['hold the steering wheel', 'rotate gripper 90 degrees', 'rotate gripper clockwise', 'rotate clockwise for 90 degrees']], # Tool and Rotation
    ["task_0049", "Rotate the steering wheel 90 degrees counterclockwise", ['hold the steering wheel', 'rotate gripper 90 degrees', 'rotate gripper counterclockwise', 'rotate counterclockwise for 90 degrees']]
    ,

    ["task_0026", "Use the gripper to push and close the drawer", ['Grasp drawer handle', 'Close the drawer', 'push in the drawer']], # Containment/Access
    ['task_0029', "Grasp the handle and open the drawer", ['Grasp drawer handle', 'Open the drawer', 'Pull out the drawer']],
    ['task_0034', "Stack the squares into a pyramid shape", ['Hold the squares', 'Stack the squares', 'Stack up a pyramid', 'Create a pyramid shape']],
    ['task_0037', "Stack the blocks in a vertical line of five", ['Hold the blocks', 'Stack the blocks', 'Stack up a line', 'Create a line of five blocks']],
    ['task_0071', "Close a sliding window", ['Grasp the window handle', 'Close the window', 'Slide the window']],

    ['task_0001', "Press the button from top to bottom", ['Push the top button first', 'Push the round button second', 'Move to the second button', 'Push the buttons in correct order']], # Sequential Actions
    ['task_0003', "Press three buttons from left to right in sequence", ['Push the left button first', 'Push the round button first', 'Aim for the second button', 'Push the buttons in correct order']],
    ['task_0023', "Turn on the power strip by pressing the button", ['Aim for the button', 'Push the top button', 'Press the button', 'Push the buttons in correct place']]


]

In [ ]:
task_command = {}
for item in full_task2command:
    task_command[item[0]] = item[2]

commands = []
labels = []
for i in task_command.keys():
    commands+= task_command[i]
    for j in range(len(task_command[i])):
        labels.append(i)

commands

In [ ]:
def label_to_task(label):
    if label in ["task_0004", "task_0006", "task_0100", "task_0111", "task_0088"]:
        return 1
    elif label in ["task_0048", "task_0049", "task_0112"]:
        return 2
    elif label in ["task_0026", "task_0029", "task_0034", "task_0037", "task_0071"]:
        return 3
    elif label in ["task_0001", "task_0003", "task_0023"]:
        return 4
labels_class = [label_to_task(i) for i in labels]
labels = [int(s.split('_')[1]) for s in labels]
labels

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
texts = commands
embeddings = model.encode(texts)
print(embeddings.shape) 

np.save("embeddings.npy", embeddings)
np.save("labels.npy", labels_class)
print("Embeddings and labels saved as .npy files.")

In [69]:
loaded_embeddings = np.load("embeddings.npy")
loaded_labels = np.load("labels.npy")

print("Embeddings and labels loaded successfully.")

Embeddings and labels loaded successfully.


In [75]:
df = pd.read_csv("VAE_UnconGaussian_train_aggregation.csv")
test_df = pd.read_csv("test_aggregation.csv")

In [77]:
df.head()

,mean_Fx,std_Fx,min_Fx,max_Fx,mean_Fy,std_Fy,min_Fy,max_Fy,mean_Fz,std_Fz,...,max_TCP_qy,mean_TCP_qz,std_TCP_qz,min_TCP_qz,max_TCP_qz,mean_TCP_qw,std_TCP_qw,min_TCP_qw,max_TCP_qw,class
0,-0.024714,0.398446,-1.803881,1.894333,-0.450330,1.046060,-7.003335,0.756397,-0.517852,1.053331,...,0.170226,-0.074916,0.252944,-0.546832,0.348799,-0.104876,0.709640,-0.981010,0.979313,4
1,-0.006140,0.775392,-3.253579,5.793850,-0.696707,1.752299,-11.823605,1.268974,-0.604465,2.075556,...,0.174449,-0.041131,0.217554,-0.329701,0.394893,-0.142525,0.692421,-0.980712,0.984883,4
2,0.015300,0.382574,-1.238802,1.441239,-0.020705,0.293863,-1.142005,1.151956,0.025054,0.249368,...,0.032133,-0.046037,0.202159,-0.469990,0.330257,0.003588,0.708966,-0.993103,0.998701,3
3,-0.013003,0.305046,-1.531037,1.383370,-0.017993,0.326706,-3.512474,1.072790,-0.008449,0.361604,...,0.008705,-0.057691,0.258708,-0.567725,0.342835,-0.003030,0.730158,-0.978676,0.978718,1
4,0.019167,0.397897,-2.063389,2.038524,-0.261686,1.241773,-15.319391,1.496653,-0.241657,1.267985,...,0.048715,-0.069650,0.257006,-0.502738,0.371942,-0.119831,0.682011,-0.980531,0.987901,1


In [79]:
test_df.head()

,mean_Fx,std_Fx,min_Fx,max_Fx,mean_Fy,std_Fy,min_Fy,max_Fy,mean_Fz,std_Fz,...,max_TCP_qy,mean_TCP_qz,std_TCP_qz,min_TCP_qz,max_TCP_qz,mean_TCP_qw,std_TCP_qw,min_TCP_qw,max_TCP_qw,class
0,0.011188,0.284069,-1.183548,1.383100,-0.312214,1.230261,-9.029436,0.954894,-0.411203,1.434380,...,0.045936,-0.085373,0.244966,-0.477972,0.366743,-0.113704,0.666684,-0.978980,0.979358,4
1,-0.028499,0.504711,-2.713423,2.219203,-0.399350,1.575871,-10.327946,1.219774,-0.526530,1.882153,...,0.072989,0.026310,0.241481,-0.525569,0.516866,0.144180,0.702940,-0.981006,0.980333,4
2,0.026818,0.948246,-2.269762,2.645966,0.002377,0.661662,-1.823600,2.395171,0.224489,0.631677,...,0.268321,0.006661,0.174156,-0.486028,0.333593,-0.120382,0.697267,-0.994079,0.995685,3
3,0.023499,2.013658,-7.825875,8.443833,0.280381,1.874367,-7.369982,7.688191,0.450387,1.583363,...,0.098862,-0.063907,0.142263,-0.437246,0.327219,0.054084,0.735278,-0.993132,0.999121,2
4,0.051888,0.374610,-1.326346,1.390408,-0.156811,0.369500,-2.814239,1.072321,-0.199171,0.365878,...,0.054578,-0.058255,0.203891,-0.435337,0.335423,-0.071685,0.678047,-0.991014,0.988604,1


In [81]:
loaded_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int64)

In [85]:
class_command = {}
for i in set(loaded_labels):
    class_command[i] = []

for i in zip(loaded_labels, loaded_embeddings):
    class_command[i[0]].append(i[1])

In [173]:
embedding_dict = class_command.copy()

all_embeddings = []
labels = []

for class_label, embedding_list in embedding_dict.items():
    for embedding in embedding_list:
        all_embeddings.append(embedding)
        labels.append(class_label)  

all_embeddings = np.array(all_embeddings)

scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(all_embeddings)

pca = PCA(n_components=5)  
embeddings_pca = pca.fit_transform(embeddings_scaled)

cols = []
for i in range(1, 6):
    cols.append(f'pca_embed{i}')
pca_df = pd.DataFrame(embeddings_pca, columns=cols)
pca_df['class'] = labels

# Step 4: Visualize the reduced embeddings
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 6))
# for class_label in pca_df['class'].unique():
#     subset = pca_df[pca_df['class'] == class_label]
#     plt.scatter(subset['PCA1'], subset['PCA2'], label=f'Class {class_label}', alpha=0.7)

# plt.title("PCA Visualization of Embedding Vectors")
# plt.xlabel("Principal Component 1")
# plt.ylabel("Principal Component 2")
# plt.legend()
# plt.grid()
# plt.show()
pca_df.head()

,pca_embed1,pca_embed2,pca_embed3,pca_embed4,pca_embed5,class
0,-11.789290,-5.186202,-2.962454,-3.832271,-7.189256,1
1,-11.225516,-5.821854,-1.871801,-2.160918,-6.767568,1
2,-11.286492,-3.916515,-1.594919,-1.369466,-4.666659,1
3,-10.834242,-3.316654,0.940967,-3.714575,-4.329308,1
4,3.966520,-2.729483,-7.512180,-0.146504,-0.877606,1


In [175]:
pca_df.sample(n=1).to_numpy()[0]

array([-7.43904257, -7.05757666, 11.5354948 ,  0.97561002, -1.56528783,
        3.        ])

In [180]:
def concatenate_embeddings(stat_features_df, embeddings_dict):
    """
    Concatenate each row in the statistical features DataFrame with a random embedding 
    from the corresponding class in the embedding dictionary.
    """
    augmented_data = []

    for _, row in stat_features_df.iterrows():
        class_label = row['class']

        class_df = pca_df[pca_df['class'] == class_label].drop('class', axis = 1)
        embedding = class_df.sample(n=1).to_numpy()[0]
        #embedding = random.choice(embeddings_dict[class_label])
        combined_features = np.concatenate([row.drop('class').values, embedding])
        augmented_data.append(combined_features)

    stat_feature_columns = [col for col in stat_features_df.columns if col != "class"]
    embedding_columns = [f"embed_{i}" for i in range(len(embedding))]
    combined_columns = stat_feature_columns + embedding_columns

    return pd.DataFrame(augmented_data, columns=combined_columns)


augmented_dataset = concatenate_embeddings(df, class_command)
augmented_dataset["class"] = df["class"].values

print("Augmented Dataset:")
print(augmented_dataset.shape)
augmented_dataset.head()

Augmented Dataset:
(9030, 58)


,mean_Fx,std_Fx,min_Fx,max_Fx,mean_Fy,std_Fy,min_Fy,max_Fy,mean_Fz,std_Fz,...,mean_TCP_qw,std_TCP_qw,min_TCP_qw,max_TCP_qw,embed_0,embed_1,embed_2,embed_3,embed_4,class
0,-0.024714,0.398446,-1.803881,1.894333,-0.450330,1.046060,-7.003335,0.756397,-0.517852,1.053331,...,-0.104876,0.709640,-0.981010,0.979313,3.169713,9.836704,1.349073,-4.635561,-0.179666,4
1,-0.006140,0.775392,-3.253579,5.793850,-0.696707,1.752299,-11.823605,1.268974,-0.604465,2.075556,...,-0.142525,0.692421,-0.980712,0.984883,2.534031,10.615719,-0.846305,-4.200302,0.213531,4
2,0.015300,0.382574,-1.238802,1.441239,-0.020705,0.293863,-1.142005,1.151956,0.025054,0.249368,...,0.003588,0.708966,-0.993103,0.998701,-0.872084,-3.794307,11.053374,5.157747,11.434239,3
3,-0.013003,0.305046,-1.531037,1.383370,-0.017993,0.326706,-3.512474,1.072790,-0.008449,0.361604,...,-0.003030,0.730158,-0.978676,0.978718,-11.286492,-3.916515,-1.594919,-1.369466,-4.666659,1
4,0.019167,0.397897,-2.063389,2.038524,-0.261686,1.241773,-15.319391,1.496653,-0.241657,1.267985,...,-0.119831,0.682011,-0.980531,0.987901,-1.973988,-1.946029,-5.351330,-6.098173,10.800629,1


In [182]:
augmented_dataset.to_csv("m3_train.csv", index=False)

In [183]:
augmented_test_dataset = concatenate_embeddings(test_df, class_command)
augmented_test_dataset["class"] = test_df["class"].values

print("Augmented Dataset:")
print(augmented_test_dataset.shape)
augmented_test_dataset.head()

Augmented Dataset:
(129, 58)


,mean_Fx,std_Fx,min_Fx,max_Fx,mean_Fy,std_Fy,min_Fy,max_Fy,mean_Fz,std_Fz,...,mean_TCP_qw,std_TCP_qw,min_TCP_qw,max_TCP_qw,embed_0,embed_1,embed_2,embed_3,embed_4,class
0,0.011188,0.284069,-1.183548,1.383100,-0.312214,1.230261,-9.029436,0.954894,-0.411203,1.434380,...,-0.113704,0.666684,-0.978980,0.979358,0.921953,9.968898,-2.710854,-3.918318,1.189810,4
1,-0.028499,0.504711,-2.713423,2.219203,-0.399350,1.575871,-10.327946,1.219774,-0.526530,1.882153,...,0.144180,0.702940,-0.981006,0.980333,3.936963,11.854113,5.641579,-2.303075,-2.992646,4
2,0.026818,0.948246,-2.269762,2.645966,0.002377,0.661662,-1.823600,2.395171,0.224489,0.631677,...,-0.120382,0.697267,-0.994079,0.995685,-0.872084,-3.794307,11.053374,5.157747,11.434239,3
3,0.023499,2.013658,-7.825875,8.443833,0.280381,1.874367,-7.369982,7.688191,0.450387,1.583363,...,0.054084,0.735278,-0.993132,0.999121,4.670001,1.791831,-4.412381,-0.983469,4.576043,2
4,0.051888,0.374610,-1.326346,1.390408,-0.156811,0.369500,-2.814239,1.072321,-0.199171,0.365878,...,-0.071685,0.678047,-0.991014,0.988604,-5.374032,-2.534800,-9.247986,-6.500840,9.929107,1


In [184]:
augmented_test_dataset.to_csv("m3_test.csv", index=False)